# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [4]:
# 1) Seu código aqui
url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'

In [6]:
r = requests.get(url)

In [7]:
r.status_code

200

In [8]:
data_json = r.json()

In [10]:
print(data_json['items'])

[{'id_pleito': 5290, 'tipo_interessado': 'Município', 'interessado': 'Casimiro de Abreu', 'cod_ibge': 3301306, 'uf': 'RJ', 'num_pvl': None, 'status': 'Deferido', 'num_processo': '17944.000078/2014-81', 'data_protocolo': '2014-11-28T02:00:00Z', 'tipo_operacao': 'Operação contratual interna', 'finalidade': 'PAC 2 - Pró-Transporte - Pavimentação e Qualificação de Vias', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'Caixa Econômica Federal', 'moeda': 'Real', 'valor': 15024627.28, 'pvl_assoc_divida': 1, 'pvl_contradado_credor': 0, 'data_status': '12/12/2014'}, {'id_pleito': 61009, 'tipo_interessado': 'Município', 'interessado': 'Rio das Flores', 'cod_ibge': 3304508, 'uf': 'RJ', 'num_pvl': 'PVL02.007214/2022-19', 'status': 'Arquivado por decurso de prazo', 'num_processo': '17944.103071/2022-20', 'data_protocolo': '2022-08-12T18:48:50Z', 'tipo_operacao': 'Operação contratual interna', 'finalidade': 'Infraestrutura', 'tipo_credor': 'Instituição Financeira Nacional', 'credor': 'C

In [15]:
df = pd.DataFrame(data_json['items'])

In [18]:
df.head(1)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,5290,Município,Casimiro de Abreu,3301306,RJ,None,Deferido,17944.000078/2014-81,2014-11-28T02:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,15024627.28,1,0,12/12/2014


In [20]:
# 2) Seu código aqui
df['status'].unique()

array(['Deferido', 'Arquivado por decurso de prazo', 'Deferido (PVL-IF)',
       'Regularizado', 'Arquivado a pedido', 'Arquivado',
       'Encaminhado à PGFN com manifestação técnica favorável',
       'Em retificação pelo interessado', 'Arquivado a pedido (PVL-IF)',
       'Arquivado pela STN', 'Em análise'], dtype=object)

In [30]:
df['status'].value_counts().nlargest(3) / len(df['status']) * 100

status
Deferido        40.540541
Arquivado       20.463320
Regularizado    10.038610
Name: count, dtype: float64

In [45]:
# 3) Seu código aqui
ano_status = df['data_status'].str.split('/').str[-1]

ano_status

0      2014
1      2023
2      2023
3      2011
4      2010
       ... 
254    2014
255    2019
256    2019
257    2017
258    2007
Name: data_status, Length: 259, dtype: object

In [48]:
# Exercicio 4
df['ano_status'].value_counts() / len(df['ano_status']) * 100

ano_status
2008    14.285714
2007    11.196911
2023    11.196911
2014     7.722008
2010     7.335907
2019     6.177606
2020     5.791506
2013     5.791506
2012     5.019305
2022     4.247104
2011     3.861004
2009     3.474903
2015     3.088803
2017     1.930502
2021     1.930502
2002     1.544402
2016     1.544402
2018     1.544402
2006     1.544402
2003     0.386100
2004     0.386100
Name: count, dtype: float64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [55]:
#1) Seu código aqui
def consultar_dados(uf, tipo_interessado):
    url = f'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf}&tipo_interessado={tipo_interessado}'
    
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data['items'])
        
        return df
    
    else:
        print(f'A requisição da API falhou. Codigo de resposta: {response.status_code}')

In [89]:
dados_consulta = consultar_dados(uf = 'GO', tipo_interessado='Estado')
dados_consulta

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,10036,Estado,Goiás,52,GO,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001105/2012-71,2012-09-21T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.500000e+09,1,0,02/10/2012
1,10592,Estado,Goiás,52,GO,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001236/2010-96,2010-10-08T03:00:00Z,Operação contratual interna (com garantia da U...,Renegociação de dívidas,Instituição Financeira Nacional,Caixa Econômica Federal,Real,3.728000e+09,0,0,08/10/2010
2,13462,Estado,Goiás,52,GO,None,Arquivado,19405.000006/2002-22,2002-02-28T03:00:00Z,Operação contratual externa (com garantia da U...,Saneamento básico,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,5.000000e+07,0,0,22/02/2006
3,24742,Estado,Goiás,52,GO,PVL02.000741/2017-27,Arquivado a pedido,17944.000426/2017-62,2017-12-29T14:06:48Z,Operação contratual interna,Aporte de capital,Instituição Financeira Nacional,Caixa Econômica Federal,Real,9.414400e+07,0,0,12/09/2019
4,11482,Estado,Goiás,52,GO,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001458/2012-71,2012-12-20T02:00:00Z,Operação contratual interna (com garantia da U...,Proinveste,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2.604160e+08,1,0,03/01/2013
5,18873,Estado,Goiás,52,GO,00000.000000/2009-62,Encaminhado à PGFN com manifestação técnica fa...,17944.000439/2015-70,2015-10-09T03:00:00Z,Operação contratual interna (com garantia da U...,Aditivo contratual,Instituição Financeira Nacional,Banco do Brasil S/A,Real,3.756584e+08,1,0,16/10/2015
6,11431,Estado,Goiás,52,GO,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001445/2012-00,2012-12-20T02:00:00Z,Operação contratual interna (com garantia da U...,Proinveste,Instituição Financeira Nacional,Caixa Econômica Federal,Real,3.669696e+08,1,0,03/01/2013
7,13438,Estado,Goiás,52,GO,None,Arquivado,19405.000001/2002-08,2002-08-07T03:00:00Z,Operação contratual externa (com garantia da U...,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Internacional,MLW Intermed Handels - und Consultinggesellsch...,Dólar dos EUA,1.000000e+07,0,0,17/12/2003
8,5176,Estado,Goiás,52,GO,None,Arquivado,17944.000052/2008-94,2008-09-30T03:00:00Z,Operação contratual interna,Assunção de Dívida,Empresa Estatal,Saneamento de Goiás,Real,1.500000e+08,0,0,30/10/2009
9,7912,Estado,Goiás,52,GO,None,Deferido,17944.000676/2007-21,2007-06-30T03:00:00Z,Concessão de garantia,Concessão de garantia a empresa estatal não de...,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.451328e+07,1,0,04/07/2007


In [60]:
# 2) Seu código aqui

dados_mg = consultar_dados(uf='MG', tipo_interessado = 'Estado')

In [87]:
df = pd.DataFrame(dados_mg)

In [84]:
df['status'].value_counts()

status
Encaminhado à PGFN com manifestação técnica favorável    20
Deferido                                                  7
Arquivado a pedido                                        5
Arquivado                                                 3
Arquivado pela STN                                        3
Arquivado por decurso de prazo                            1
Indeferido                                                1
Name: count, dtype: int64

In [85]:
df['status'].value_counts().get('Arquivado por decurso de prazo', 0)

1

In [93]:
# 3) Seu código aqui
dados_ba = consultar_dados(uf='BA', tipo_interessado='Município')

In [97]:
df = pd.DataFrame(dados_ba)
df.head(1)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,60731,Município,Barra da Estiva,2902807,BA,PVL02.007122/2022-21,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Agência de Fomento do Estado da Bahia S/A,Real,4000000.0,0,1,15/07/2022


In [110]:
deferidas = df[df['status'] == 'Deferido']

In [111]:
deferidas['interessado'].value_counts()

interessado
Luís Eduardo Magalhães    16
Vitória da Conquista      12
Barreiras                 10
Camaçari                  10
Lauro de Freitas          10
                          ..
Malhada                    1
Sobradinho                 1
Cafarnaum                  1
Itapicuru                  1
Itagibá                    1
Name: count, Length: 185, dtype: int64

In [112]:
# 4) Seu código aqui
dados_ba_csv = consultar_dados(uf='BA', tipo_interessado='Estado')

In [114]:
dados_ba_csv.to_csv('dados_bahia.csv', index= False, sep=';', decimal=',')